In [160]:
# !pip install pandas
# !pip install numpy
# !pip install plotly
# !pip install networkx
# !pip install matplotlib

In [161]:
# importing module
import pandas as pd
import numpy as np
# # dataset
# dataset = pd.read_csv("Market_Basket_Optimisation.csv")
# # printing the shape of the dataset
# dataset.shape

In [162]:
# # importing module
# import numpy as np
# # Gather All Items of Each Transactions into Numpy Array
# transaction = []
# for i in range(0, dataset.shape[0]):
#     for j in range(0, dataset.shape[1]):
#         transaction.append(dataset.values[i,j])
# # converting to numpy array
# transaction = np.array(transaction)
# # saving the transaction array
# np.save('transaction.npy', transaction)
# print(transaction)

In [163]:
# loading from teh temporary file
transaction = np.load('transaction.npy')

In [164]:
#  Transform Them a Pandas DataFrame
df = pd.DataFrame(transaction, columns=["items"]) 
# Put 1 to Each Item For Making Countable Table, to be able to perform Group By
df["incident_count"] = 1 
#  Delete NaN Items from Dataset
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)
# Making a New Appropriate Pandas DataFrame for Visualizations  
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
#  Initial Visualizations
df_table.head(5).style.background_gradient(cmap='Blues')

,items,incident_count
0,mineral water,1787
1,eggs,1348
2,spaghetti,1306
3,french fries,1282
4,chocolate,1230


In [165]:
# importing required module
import plotly.express as px
# to have a same origin
df_table["all"] = "Top 50 items" 
# creating tree map using plotly
fig = px.treemap(df_table.head(50), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(50), hover_data=['items'],
                  color_continuous_scale='Blues',
                )
# ploting the treemap
fig.show()

In [166]:
# #  Transform Every Transaction to Seperate List & Gather Them into Numpy Array
# transaction = []
# for i in range(dataset.shape[0]):
#     transaction.append([str(dataset.values[i,j]) for j in range(dataset.shape[1])])
# # creating the numpy array of the transactions
# transaction = np.array(transaction)

# # saving the transaction array
# np.save('transaction1.npy', transaction)

In [167]:
# loading from teh temporary file
transaction = np.load('transaction1.npy')

In [168]:
# importing the required module
from mlxtend.preprocessing import TransactionEncoder
# initializing the transactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
# dataset after encoded
dataset.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [169]:
# select top 30 items
first30 = df_table["items"].head(30).values 
# Extract Top 30
dataset = dataset.loc[:,first30] 
# shape of the dataset
dataset.shape

(7500, 30)

In [170]:
from mlxtend.frequent_patterns import apriori, association_rules

# Extracting the most frequest itemsets via Mlxtend.
# The length column has been added to increase ease of filtering.
frequent_itemsets = apriori(dataset, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
# printing the frequent itemset
frequent_itemsets

,support,itemsets,length
0,0.238267,(mineral water),1
1,0.179733,(eggs),1
2,0.174133,(spaghetti),1
3,0.170933,(french fries),1
4,0.163867,(chocolate),1
...,...,...,...
205,0.010933,"(ground beef, chocolate, mineral water)",3
206,0.011067,"(milk, ground beef, mineral water)",3
207,0.011067,"(milk, frozen vegetables, mineral water)",3
208,0.010533,"(eggs, spaghetti, chocolate)",3


In [171]:
# printing the frequntly items 
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.05) ]

,support,itemsets,length
30,0.050933,"(eggs, mineral water)",2
31,0.059733,"(spaghetti, mineral water)",2
33,0.052667,"(chocolate, mineral water)",2


In [172]:
# printing the frequntly items with length 3
frequent_itemsets[ (frequent_itemsets['length'] == 3) ].head(3)

,support,itemsets,length
193,0.014267,"(eggs, spaghetti, mineral water)",3
194,0.013467,"(eggs, chocolate, mineral water)",3
195,0.013067,"(eggs, milk, mineral water)",3


In [173]:
#  We set our metric as "Lift" to define whether antecedents & consequents are dependent our not
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
211,(herb & pepper),(ground beef),0.049467,0.098267,0.016000,0.323450,3.291555,0.011139,1.332841,1,1
210,(ground beef),(herb & pepper),0.098267,0.049467,0.016000,0.162822,3.291555,0.011139,1.135402,1,1
286,(ground beef),"(spaghetti, mineral water)",0.098267,0.059733,0.017067,0.173677,2.907540,0.011197,1.137893,1,2
283,"(spaghetti, mineral water)",(ground beef),0.059733,0.098267,0.017067,0.285714,2.907540,0.011197,1.262427,2,1
304,(olive oil),"(spaghetti, mineral water)",0.065733,0.059733,0.010267,0.156187,2.614731,0.006340,1.114306,1,2
...,...,...,...,...,...,...,...,...,...,...,...
55,(low fat yogurt),(eggs),0.076400,0.179733,0.016800,0.219895,1.223453,0.003068,1.051483,1,1
114,(escalope),(french fries),0.079333,0.170933,0.016400,0.206723,1.209376,0.002839,1.045116,1,1
115,(french fries),(escalope),0.170933,0.079333,0.016400,0.095944,1.209376,0.002839,1.018373,1,1
156,(shrimp),(green tea),0.071333,0.132000,0.011333,0.158879,1.203625,0.001917,1.031956,1,1


In [174]:
# Sort values based on confidence
rules.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
260,"(eggs, ground beef)",(mineral water),0.020000,0.238267,0.010133,0.506667,2.126469,0.005368,1.544054,2,1
318,"(milk, ground beef)",(mineral water),0.022000,0.238267,0.011067,0.503030,2.111207,0.005825,1.532756,2,1
312,"(ground beef, chocolate)",(mineral water),0.023067,0.238267,0.010933,0.473988,1.989319,0.005437,1.448130,2,1
324,"(milk, frozen vegetables)",(mineral water),0.023600,0.238267,0.011067,0.468927,1.968075,0.005444,1.434328,2,1
34,(soup),(mineral water),0.050533,0.238267,0.023067,0.456464,1.915771,0.011026,1.401441,1,1
...,...,...,...,...,...,...,...,...,...,...,...
323,(mineral water),"(milk, ground beef)",0.238267,0.022000,0.011067,0.046447,2.111207,0.005825,1.025637,1,2
317,(mineral water),"(ground beef, chocolate)",0.238267,0.023067,0.010933,0.045887,1.989319,0.005437,1.023918,1,2
305,(mineral water),"(spaghetti, olive oil)",0.238267,0.022933,0.010267,0.043089,1.878880,0.004802,1.021063,1,2
265,(mineral water),"(eggs, ground beef)",0.238267,0.020000,0.010133,0.042529,2.126469,0.005368,1.023530,1,2


# Custom Implementation

In [175]:
# Using conviction
rules = association_rules(frequent_itemsets, metric="conviction", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("conviction",ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
19,"(eggs, ground beef)",(mineral water),0.020000,0.238267,0.010133,0.506667,2.126469,0.005368,1.544054,2,1
35,"(milk, ground beef)",(mineral water),0.022000,0.238267,0.011067,0.503030,2.111207,0.005825,1.532756,2,1
34,"(ground beef, chocolate)",(mineral water),0.023067,0.238267,0.010933,0.473988,1.989319,0.005437,1.448130,2,1
36,"(milk, frozen vegetables)",(mineral water),0.023600,0.238267,0.011067,0.468927,1.968075,0.005444,1.434328,2,1
26,"(ground beef, mineral water)",(spaghetti),0.040933,0.174133,0.017067,0.416938,2.394361,0.009939,1.416431,2,1
6,(soup),(mineral water),0.050533,0.238267,0.023067,0.456464,1.915771,0.011026,1.401441,1,1
29,"(pancakes, spaghetti)",(mineral water),0.025200,0.238267,0.011467,0.455026,1.909736,0.005462,1.397744,2,1
31,"(spaghetti, olive oil)",(mineral water),0.022933,0.238267,0.010267,0.447674,1.878880,0.004802,1.379138,2,1
9,(ground beef),(spaghetti),0.098267,0.174133,0.039200,0.398915,2.290857,0.022088,1.373959,1,1
22,"(spaghetti, milk)",(mineral water),0.035467,0.238267,0.015733,0.443609,1.861817,0.007283,1.369061,2,1


In [176]:
# Leverage is not found in apriori algorithm, so only conviction is implemented